In [1]:

"""Complete LSTM-ENSEMBLE Implementation with Rank Reduction"""

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import LSTM, Dense, Input, Concatenate, Multiply, Lambda, Softmax
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from scipy.signal import savgol_filter, welch
import tensorflow as tf
from scipy.linalg import svd
from tensorflow.keras.layers import Layer


# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# ================== UTILITY FUNCTIONS ==================
def load_data(file_path):
    """Load acceleration data from file"""
    data = pd.read_csv(file_path, sep='\s+', header=None, skiprows=1)
    return data[1].values

def create_sequences(data, delay_params):
    """Generate multi-rate sequences"""
    sequences = {}
    for tau, d in delay_params:
        X, y = [], []
        for i in range(len(data) - d*tau - 1):
            X.append(data[i:i+d*tau:tau])
            y.append(data[i + d*tau])
        X = np.array(X).reshape(-1, d, 1)
        sequences[(tau, d)] = (X, np.array(y))
        print(f"Generated {(tau,d)}: {X.shape[0]} samples")
    return sequences

def align_sequences(sequences_dict, delay_params):
    """Ensure equal sample counts across all branches"""
    min_samples = min(sequences_dict[(tau, d)][0].shape[0] for (tau, d) in delay_params)
    X = [sequences_dict[(tau, d)][0][:min_samples] for (tau, d) in delay_params]
    y = sequences_dict[delay_params[0]][1][:min_samples]
    return X, y
class RankReducedLSTM(Layer):

    def __init__(self, units, reduction_ratio=0.8, return_sequences=False, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.reduction_ratio = reduction_ratio
        self.return_sequences = return_sequences
        self.state_size = [units, units]
        self.output_size = units

    def build(self, input_shape):
        input_dim = input_shape[-1]
        total_dim = input_dim + self.units

        # Original weight matrix
        self.W = self.add_weight(
            shape=(total_dim, 4 * self.units),
            name='kernel',
            initializer='glorot_uniform')

        self.bias = self.add_weight(
            shape=(4 * self.units,),
            name='bias',
            initializer='zeros')

        # Compression matrices
        self.W_basis = None
        self.C = None
        self.compressed = False

    def call(self, inputs, initial_state=None):

        # DEBUG LINE
        print(f"\n[DEBUG] Layer '{self.name}' - Compressed: {self.compressed}")
        if self.compressed:
            print(f"[DEBUG] Using compressed path (rank={self.reduced_rank})")
        else:
            print(f"[DEBUG] Using standard (uncompressed) path")
        # Initialize states
        if initial_state is None:
            batch_size = tf.shape(inputs)[0]
            h_prev = tf.zeros((batch_size, self.units), dtype=inputs.dtype)
            c_prev = tf.zeros((batch_size, self.units), dtype=inputs.dtype)
        else:
            h_prev, c_prev = initial_state

        # Prepare inputs - ensure 2D shape [batch_size, features]
        if len(inputs.shape) == 3:
            inputs = inputs[:, -1, :]  # Take last timestep if sequence

        # Concatenate input and hidden state
        z = tf.concat([inputs, h_prev], axis=-1)

        if self.compressed:
            # Compressed forward pass
            r = tf.shape(self.W_basis)[0]  # reduced rank
            z1 = z[:, :r]  # basis components
            z2 = z[:, r:]  # dependent components

            # Project dependent components
            z2_coef = tf.matmul(z2, self.C)

            # Combine and project through basis
            z_reduced = z1 + z2_coef
            z_out = tf.matmul(z_reduced, self.W_basis) + self.bias
        else:
            # Standard forward pass
            z_out = tf.matmul(z, self.W) + self.bias

        # Gates computation
        i, f, c, o = tf.split(z_out, 4, axis=-1)

        # LSTM operations
        c_new = tf.sigmoid(f) * c_prev + tf.sigmoid(i) * tf.tanh(c)
        h_new = tf.sigmoid(o) * tf.tanh(c_new)

        if self.return_sequences:
            return tf.expand_dims(h_new, axis=1), [h_new, c_new]
        return h_new, [h_new, c_new]

    def compute_output_shape(self, input_shape):
        if self.return_sequences:
            return (input_shape[0], 1, self.units)
        return (input_shape[0], self.units)

    def compress(self):
        """Apply verified rank reduction"""
        W = self.W.numpy()
        m, n = W.shape
        # Calculate actual reduced rank
        self.reduced_rank = max(1, int(m * self.reduction_ratio))


        #DEBUG LINE
        print(f"\n[DEBUG] Compressing layer '{self.name}'")
        print(f"Original weight shape: {W.shape}, reduction_ratio={self.reduction_ratio}")
        print(f"Target reduced rank: {self.reduced_rank}")
        # SVD decomposition
        U, s, Vh = svd(W, full_matrices=False)

        # Truncated matrices
        Ur = U[:, :self.reduced_rank]
        Sr = np.diag(s[:self.reduced_rank])
        Vhr = Vh[:self.reduced_rank, :]

        # Low-rank approximation
        W_r = Ur @ Sr @ Vhr

        # RECONSTRUCTION ERROR CHECK
        reconstruction_error = np.linalg.norm(W - W_r)
        print(f"Reconstruction error: {reconstruction_error:.2e}")

        # Split into basis and dependent rows
        W_basis = W_r[:self.reduced_rank, :]
        W_dep = W_r[self.reduced_rank:, :]

        # Compute projection matrix
        C = W_dep @ np.linalg.pinv(W_basis)

      #  NORM CHECK
        print(f"W_basis norm: {np.linalg.norm(W_basis):.4f}")
        print(f"C matrix norm: {np.linalg.norm(C):.4f}")

        # Store as TF variables
        self.W_basis = tf.Variable(W_basis, dtype=tf.float32, trainable=False)
        self.C = tf.Variable(C, dtype=tf.float32, trainable=False)
        self.compressed = True
        self.trainable = False

        # Calculate compression statistics
        original_params = m * n
        compressed_params = self.reduced_rank * (n + m - self.reduced_rank)
        compression_ratio = compressed_params / original_params

        print(f"Compressed LSTM from {W.shape} to rank {self.reduced_rank} "
              f"(Param reduction: {1-compression_ratio:.1%})")

# ================== MAIN PIPELINE ==================
def main():
    total_start_time = time.time()

    # === DATA LOADING & PREPROCESSING ===
    data_pretrain = load_data('/content/drive/MyDrive/acc_base_a0.txt')
    data_finetune = load_data('/content/drive/MyDrive/acc_base_a38.txt')

    scaler = StandardScaler()
    data_pretrain = scaler.fit_transform(data_pretrain.reshape(-1, 1)).flatten()
    data_finetune = scaler.transform(data_finetune.reshape(-1, 1)).flatten()

    # === MULTI-RATE SAMPLING ===
    delay_params = [(2,15), (5,14), (8,12), (11,10), (14,8)]
    pretrain_sequences = create_sequences(data_pretrain, delay_params)
    finetune_sequences = create_sequences(data_finetune, delay_params)

    # === DATA ALIGNMENT ===
    X_pretrain, y_pretrain = align_sequences(pretrain_sequences, delay_params)
    X_finetune, y_finetune = align_sequences(finetune_sequences, delay_params)

    # === MODEL ARCHITECTURE ===
    COMMON_DIM = 32
    inputs = [Input(shape=(d, 1)) for (tau, d) in delay_params]

    # LSTM branches with projection and rank reduction
    lstm_outputs = []
    for inp, (tau, d) in zip(inputs, delay_params):
        lstm_layer = RankReducedLSTM(
            units=2*d,
            reduction_ratio=0.9,  # 20% reduction
            return_sequences=False
        )
        x = lstm_layer(inp)
        x = Dense(COMMON_DIM, activation='tanh')(x)
        lstm_outputs.append(x)

    # Attention mechanism
    attention = Dense(len(delay_params), activation='tanh')(Concatenate()(lstm_outputs))
    #attention_weights = Softmax(axis=1)(attention)
    #attention_weights = Lambda(lambda x: tf.expand_dims(x, -1))(attention_weights)
    # --- Attention weights (batch, branches) → (batch, branches, 1)
    attention_weights = Softmax(axis=1)(attention)
    attention_weights = Lambda(
       lambda x, _tf=tf:_tf.expand_dims(x, axis=-1),
       output_shape=lambda s: (s[0], s[1], 1)
      )(attention_weights)

# --- Expand each branch (batch, COMMON_DIM) → (batch, 1, COMMON_DIM)
    expanded = [
      Lambda(
        lambda x , _tf=tf:_tf.expand_dims(x, axis=1),
        output_shape=lambda s: (s[0], 1, s[1])
      )(o)
       for o in lstm_outputs
       ]

# --- Multiply & sum
    concatenated = Concatenate(axis=1)(expanded)
    weighted_output = Multiply()([concatenated, attention_weights])

# --- Sum over the branches dimension
    summed = Lambda(
    lambda x, _tf=tf: _tf.reduce_sum(x, axis=1),
    output_shape=lambda s: (s[0], s[2])
    )    (weighted_output)

# --- Final prediction
    output = Dense(1)(summed)



    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.006), loss='mse')

    # Submodel for attention weights extraction
    attention_model = Model(inputs=model.inputs, outputs=attention_weights)

    # === TRAINING ===
    early_stop = EarlyStopping(monitor='val_loss', patience=5)

    print("\n=== Pretraining ===")
    pretrain_history = model.fit(
        X_pretrain, y_pretrain,
        epochs=50,
        batch_size=32,
        validation_split=0.1,
        callbacks=[early_stop]
    )

    print("\n=== Fine-tuning ===")
    finetune_history = model.fit(
        X_finetune, y_finetune,
        epochs=20,
        batch_size=32,
        validation_split=0.1,
        callbacks=[early_stop]
    )
       # === ORIGINAL (UNCOMPRESSED) EVALUATION ===
    y_full = model.predict(X_finetune).flatten()
    #y_full_smooth = savgol_filter(y_full, window_length=5, polyorder=2)
    # === UNCOMPRESSED EVALUATION & TIMING ===
    print("\n--- Uncompressed model ---")
    mse_un = mean_squared_error(y_finetune, y_full)
    mae_un = mean_absolute_error( y_finetune, y_full)
    r2_un  = r2_score(          y_finetune, y_full)

    # Batch-mode timing
    t0 = time.perf_counter()
    _  = model.predict(X_finetune, verbose=0)
    t1 = time.perf_counter()
    ms_per_sample_un = (t1 - t0) * 1e3 / len(X_finetune)

    print(f"MSE: {mse_un:.4f}  MAE: {mae_un:.4f}  R²: {r2_un:.4f}")
    print(f"Avg inference: {ms_per_sample_un:.3f} ms/sample")

   # === MODEL COMPRESSION ===
    print("\n=== Model Compression ===")
    for layer in model.layers:
        if isinstance(layer, RankReducedLSTM):
            layer.compress()
        #  POST-COMPRESSION CHECK
        #print(f"[POST-COMPRESS] Layer '{layer.name}' compressed={layer.compressed}")
        #print(f"  W_basis shape: {layer.W_basis.shape}")
        #print(f"  C shape: {layer.C.shape}\n")
    # === COMPRESSED EVALUATION & TIMING ===
    print("\n--- Compressed model ---")
    y_cp = model.predict(X_finetune, verbose=0).flatten()
    mse_cp = mean_squared_error(y_finetune, y_cp)
    mae_cp = mean_absolute_error( y_finetune, y_cp)
    r2_cp  = r2_score(          y_finetune, y_cp)

    t0 = time.perf_counter()
    _  = model.predict(X_finetune, verbose=0)
    t1 = time.perf_counter()
    ms_per_sample_cp = (t1 - t0) * 1e3 / len(X_finetune)

    print(f"MSE: {mse_cp:.4f}  MAE: {mae_cp:.4f}  R²: {r2_cp:.4f}")
    print(f"Avg inference: {ms_per_sample_cp:.3f} ms/sample")

    # === SPEED-UP & ERROR CHANGE ===
    speedup = ms_per_sample_un / ms_per_sample_cp
    print(f"\nSpeed-up: {speedup:.2f}× faster after compression")
    print(f"ΔMSE: {mse_cp - mse_un:+.2e}, ΔMAE: {mae_cp - mae_un:+.2e}")

    print("\n=== Verifying Compression Status ===")
    for layer in model.layers:
        if isinstance(layer, RankReducedLSTM):
           print(f"Layer {layer.name}: compressed={layer.compressed}, trainable={layer.trainable}")



    # === TIMING BENCHMARK (per‐sample & per‐timestep) ===
    # Warm‐up
     = model.predict([x[:1] for x in X_finetune], verbose=0)
     # === TIMING BENCHMARK(UPDATED TO USE compressed_model)
    times = []
    for i in range(len(X_finetune[0])):
       sample = [x[i:i+1] for x in X_finetune]
       t0 = time.perf_counter()
       model.predict(sample, verbose=0)  # Changed from model to compressed_model
       t1 = time.perf_counter()
       times.append((t1 - t0) * 1e3)

    mean_ms = np.mean(times)
    seq_len = X_finetune[0].shape[1]
    per_step_us = mean_ms * 1e3 / seq_len  # convert ms→μs then divide

    print(f"\nAvg inference time: {mean_ms:.3f} ms per sample")
    print(f"≈ {per_step_us:.2f} μs per timestep (sequence length = {seq_len})")
    print(f"\\n✅ Total run time: {time.time() - total_start_time:.2f}s")

if __name__ == "__main__":
    main()

Mounted at /content/drive
Generated (2, 15): 1408 samples
Generated (5, 14): 1368 samples
Generated (8, 12): 1342 samples
Generated (11, 10): 1328 samples
Generated (14, 8): 1326 samples
Generated (2, 15): 1408 samples
Generated (5, 14): 1368 samples
Generated (8, 12): 1342 samples
Generated (11, 10): 1328 samples
Generated (14, 8): 1326 samples

=== Pretraining ===
Epoch 1/50

[DEBUG] Layer 'rank_reduced_lstm_4' - Compressed: False
[DEBUG] Using standard (uncompressed) path

[DEBUG] Layer 'rank_reduced_lstm_3' - Compressed: False
[DEBUG] Using standard (uncompressed) path

[DEBUG] Layer 'rank_reduced_lstm_2' - Compressed: False
[DEBUG] Using standard (uncompressed) path

[DEBUG] Layer 'rank_reduced_lstm_1' - Compressed: False
[DEBUG] Using standard (uncompressed) path

[DEBUG] Layer 'rank_reduced_lstm' - Compressed: False
[DEBUG] Using standard (uncompressed) path

[DEBUG] Layer 'rank_reduced_lstm_4' - Compressed: False
[DEBUG] Using standard (uncompressed) path

[DEBUG] Layer 'rank_r